# PyTorch로 딥러닝하기 : 60분만에 끝장내기

> 이 튜토리얼의 목표:
- 높은 수준에서 PyTorch의 Tensor library와 신경망(Neural Network)를 이해합니다.
- 이미지를 분류하는 작은 신경망을 학습시킵니다.

# PyTorch란?

> Python 기반의 과학 연산 패키지로 다음과 같은 두 집단을 대상으로 합니다
- NumPy를 대체하고 GPU의 연산력을 사용
- 최대한의 유연성과 속도를 제공하는 딥러닝 연구 플랫폼

## Tensors

- Tensor는 NumPy의 ndarray와 유사할뿐만 아니라, GPU를 사용한 연산 가속도 지원합니다.

In [ ]:
from __future__ import print_function
import torch

# 초기화되지 않은 5x3 행렬 생성
x1 = torch.Tensor(5,3)
print(x1)

# 무작위로 초기화된 행렬을 생성
x2 = torch.rand(5,3)
print(x2)

# 행렬의 크기 
print(x1.size())

## 연산(Operations)

https://pytorch.org/docs/stable/torch.html

In [ ]:
# 덧셈
x = torch.rand(5,3)
y = torch.rand(5,3)
print(x + y)
print(torch.add(x,y))

# 결과 tensor를 인자로 제공
result = torch.Tensor(5,3)
torch.add(x, y, out=result)
print(result)

# in-place
# y에 x 더하기 (y가 바뀜)
y.add_(x)
print(y)

In [ ]:
# 인덱싱
print(x[:, 1])

In [ ]:
# 크기 변경 : tensor의 크기(size)나 모양(shape)을 변경
x = torch.randn(4,4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(), z.size())

## NumPy 변환(Bridge)

Torch Tensor를 NumPy 배열(array)로 변환하거나, 그 반대로 하는 것은 매우 쉽습니다.

Torch Tensor와 NumPy 배열은 저장 공간을 공유하기 때문에, 하나를 변경하면 다른 하나도 변경됩니다.

CharTensor를 제외한 CPU 상의 모든 Tensor는 NumPy로의 변환을 지원하며, (NumPy에서 Tensor로의) 반대 변환도 지원합니다.

In [ ]:
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

a.add_(1) # a(torch tensor)만 변경했는데 b(np array)도 변경
print(a)
print(b)

In [ ]:
# np 배열을 변경하면 Torch Tensor 값도 자동 변경
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

## CUDA Tensors

.cuda 메소드를 사용하여 Tensor를 GPU 상으로 옮길 수 있습니다.

In [ ]:
# 이 코드는 CUDA가 사용 가능한 환경에서만 실행
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x+y

# Autograd : 자동 미분

PyTorch의 모든 신경망의 중심에는 autograd 패키지가 있습니다. 먼저 이것을 가볍게 살펴본 뒤, 첫번째 신경망을 학습시켜보겠습니다.

autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다. 

이는 실행-기반-정의(define-by-run) 프레임워크로, **이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻**이며, 역전파는 학습 과정의 매 단계마다 달라집니다.

## 변수(Variable)

> autograd.Variable 클래스
- 패키지의 중심 클래스
- 이는 Tensor를 감싸고(wrap) 있으며, Tensor 기반으로 정의된 거의 대부분의 연산을 지원합니다. 
- 계산이 완료된 후 .backward() 를 호출하여 모든 변화도(gradient)을 자동으로 계산할 수 있습니다.

- .data 속성을 사용하여 tensor 자체(raw tensor)에 접근할 수 있으며, 이 변수와 관련된 변화도는 .grad 에 누적됩니다.


> Function 클래스
- 또한 Autograd 구현에서 매우 중요한 클래스
- Variable 과 Function 은 상호 연결되어 있으며, 모든 연산 과정을 부호화(encode)하여 순환하지 않은 그래프(acyclic graph)를 생성합니다. 
- 각 변수는 .grad_fn 속성을 갖고 있는데, 이는 Variable 을 생성한 Function 을 참조하고 있습니다. (단, 사용자가 만든 Variable은 예외로, 이 때 grad_fn 은 None 입니다.)

- 도함수를 계산하기 위해서는, Variable 의 .backward() 를 호출하면 됩니다. 
- Variable 이 스칼라(scalar)인 경우(예. 하나의 요소만 갖는 등)에는, backward 에 인자를 정해줄 필요가 없습니다. 
- 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 gradient 의 인자로 지정할 필요가 있습니다.

In [ ]:
import torch
from torch.autograd import Variable

# 변수 생성
x = Variable(torch.ones(2,2), requires_grad=True)
print(x)

# 변수에 연산 수행
y = x + 2
print(y)

# y는 연산의 결과로 생성된 것이므로, grad_fn을 갖음
print(y.grad_fn)

# y에 다른 연산 수행
z = y * y * 3
out = z.mean()

print(z, out)

## 변화도(Gradient)

이제 역전파(backprop)를 해보겠습니다. 

out.backward() 는 out.backward(torch.Tensor([1.0])) 를 하는 것과 똑같습니다.

In [ ]:
out.backward()

4.5 로 이루어진 행렬이 보일 것입니다. 
out 을 변수 “o” 라고 하면, 다음과 같이 구할 수 있습니다. $$o=\frac{1}{4}\sum_{i}{z_i}$$
$$z_i=3(x_i+2)^2$$
$$z_i\bigr\rvert_{x_i=1} = 27$$ 

$$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$$
$$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$$

In [ ]:
# 변화도 d(out)/dx를 출력
print(x.grad)

In [ ]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

In [ ]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

# 신경망(Neural Networks)

신경망은 torch.nn 패키지를 사용하여 생성할 수 있습니다.

지금까지 autograd 를 살펴봤는데요, nn 은 모델을 정의하고 미분하는데 autograd 를 사용합니다. 

nn.Module 은 계층(layer)과 output 을 반환하는 forward

*   항목 추가
*   항목 추가

(input) 메서드를 포함하고 있습니다.


숫자 이미지를 분류하는 신경망을 예제로 살펴보겠습니다

이는 간단한 피드-포워드 네트워크(Feed-forward network)입니다. 

입력(input)을 받아 여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공합니다.

> 신경망의 전형적인 학습 과정은 다음과 같습니다:
- 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.
- 데이터셋(dataset) 입력을 반복합니다.
- 입력을 신경망에서 처리합니다.
- 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.
-  신경망의 가중치를 갱신합니다. 

- 일반적으로 다음과 같은 간단한 규칙을 사용합니다:
- 가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)

## 신경망 정의

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

forward 함수만 정의하게 되면, (변화도를 계산하는) backward 함수는 autograd 를 사용하여 자동으로 정의됩니다. 

forward 함수에서는 어떠한 Tensor 연산을 사용해도 됩니다.

모델의 학습 가능한 매개변수들은 net.parameters() 에 의해 반환됩니다.

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

forward의 입력은 autograd.Variable 이고, 출력 또한 마찬가지입니다. 

Note: 이 신경망(LeNet)의 입력은 32x32입니다. 

이 신경망에 MNIST 데이터셋을 사용하기 위해서는, 데이터셋의 이미지를 32x32로 크기를 변경해야 합니다.

In [ ]:
input = Variable(torch.randn(1,1,32,32))
out = net(input)
print(out)

모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 합니다:

In [ ]:
net.zero_grad()
out.backward(torch.randn(1,10))

torch.nn 은 미니 배치(mini-batch)만 지원합니다. 

torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니배치만을 입력으로 받습니다.

예를 들어, nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, input.unsqueeze(0) 을 사용해서 가짜 차원을 추가합니다.

계속 진행하기 전에, 지금까지 살펴봤던 것들을 다시 한번 요약해보겠습니다.

> Recap:
- torch.Tensor - 다차원 배열.
- autograd.Variable - Tensor를 감싸고 모든 연산을 기록 합니다. Tensor 와 동일한 API를 갖고 있으며, backward() 와 같이 추가된 것들도 있습니다. 또한, tensor에 대한 변화도를 갖고 있습니다.
- nn.Module - 신경망 모듈. 매개변수를 캡슐화(Encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공합니다.
- nn.Parameter - 변수의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록 됩니다.
- autograd.Function - autograd 연산의 전방향과 역방향 정의 를 구현합니다. 모든 Variable 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Variable 을 생성하고 이력(History)을 부호화 하는 함수들과 연결하고 있습니다.

이 시점에 우리가 다룬 내용은 다음과 같습니다:
- 신경망을 정의하는 것
- 입력을 처리하고 backward 를 호출하는 것

더 살펴볼 내용들은 다음과 같습니다:
- 손실을 계산하는 것
- 신경망의 가중치를 갱신하는 것

## 손실 함수(Loss Function)

손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 떨어져있는지를 추정하는 값을 계산합니다.

nn 패키지에는 여러가지의 손실 함수들 이 존재합니다. 간단한 손실 함수로는 출력과 대상간의 평균자승오차(mean-squared error)를 계산하는 nn.MSEloss 가 있습니다.

In [ ]:
# 예를 들면
output = net(input)
target = Variable(torch.arange(1,11)) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output.to(torch.float32), target.to(torch.float32))
print(loss)

이제 .grad_fn 속성을 사용하여 loss 를 역방향에서 따라가다보면, 이러한 모습의 연산 그래프를 볼 수 있습니다.

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

      
우리가 loss.backward() 를 실행할 때, 전체 그래프는 손실에 대해 미분되며, 그래프 내의 모든 변수는 변화도가 누적된 .grad 함수를 갖게 됩니다.

In [ ]:
# 몇 단계만
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

## 역전파(Backprop)

오차(error)를 역전파하기 위해 할 일은 loss.backward() 이 전부입니다. 

기존 변화도를 지우는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에 누적되기 때문입니다.

이제 loss.backward() 를 호출하여 역전파 전과 후에 conv1의 bias gradient를 살펴보겠습니다.

In [ ]:
loss

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

## 가중치 갱신

실제로 많이 사용되는 가장 단순한 갱신 규칙은 확률적 경사하강법(SGD; Stochastic Gradient Descent)입니다:
- 가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

그러나, 신경망을 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 갱신 규칙을 사용하고 싶을 수 있습니다. 

이를 위해서 torch.optim 라는 작은 패키지에 이러한 방법들을 모두 구현해두었습니다. 사용법은 매우 간단합니다:

In [ ]:
import torch.optim as optim

# Optimizer를 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training Loop)에서는 다음과 같음
optimizer.zero_grad() # zero the gradient buffers 수동으로 변화도 버퍼를 0으로 설정(변화도 누적 방지)
output = net(input)
loss = criterion(output.to(torch.float32), target.to(torch.float32))
loss.backward() 
optimizer.step() # Does the update


# 분류기(Classifier) 학습하기

일반적으로 이미지나 텍스트, 오디오나 비디오 데이터를 다룰텐데요, 이러한 데이터는 표준 Python 패키지를 사용하여 불러온 후 NumPy 배열로 변환하면 됩니다. 그리고 그 배열을 torch.*Tensor 로 변환하면 됩니다.

- 이미지는 Pillow나 OpenCV 같은 패키지가 유용합니다.
- 오디오를 처리할 때는 SciPy와 LibROSA가 유용하고요.
- 텍스트의 경우에는 그냥 Python이나 Cython의 것들을 사용하거나, NLTK나 SpaCy도 좋습니다.

특별히 영상 분야를 위해서는 torchvision 이라는 패키지를 만들어두었는데요, 여기에는 Imagenet이나 CIFAR10, MNIST 등과 같은 일반적으로 사용하는 데이터셋을 불러오는 함수들(data loaders)이나, image, viz., torchvision.datasets 와 torch.utils.data.DataLoader 데이터 변환기가 포함되어 있습니다.

이러한 기능은 엄청나게 편리하며, 매번 유사한 코드(boilerplate code)를 반복해서 작성하는 것을 피할 수 있습니다.

이 튜토리얼에서는 CIFAR10 데이터셋을 사용할 텐데요, 여기에는 다음과 같은 분류들이 있습니다: 
- ‘비행기(airplane)’, ‘자동차(automobile)’, ‘새(bird)’, ‘고양이(cat)’, ‘사슴(deer)’, ‘개(dog)’, ‘개구리(frog)’, ‘말(horse)’, ‘배(ship)’, ‘트럭(truck)’. 

그리고 CIFAR10에 포함된 이미지의 크기는 3x32x32인데요, 이는 32x32 픽셀 크기의 이미지가 3개 채널(channel)로 이뤄져 있다는 뜻입니다.

## 이미지 분류기 학습

다음의 단계로 진행해보겠습니다:

1. CIFAR10의 학습용 / 시험(test)용 데이터셋을 torchvision 을 사용하여 불러오고, 정규화(nomarlizing)합니다.
2. 합성곱 신경망(Convolution Neural Network)을 정의합니다.
3. 손실 함수를 정의합니다.
4. 학습용 데이터를 사용하여 신경망을 학습합니다.
5. 시험용 데이터를 사용하여 신경망을 검사합니다.

## CIFAR10를 불러오고 정규화하기

In [ ]:
# torchvision을 사용하면 매우 쉽게 CIFAR10 데이터를 불러올 수 있음
import torch
import torchvision
import torchvision.transforms as transforms

torchvision 데이터셋의 출력(output)은 [0, 1] 범위를 갖는 PILImage 이미지입니다. 

이를 [-1, 1]의 범위로 정규화된 Tensor로 변환하겠습니다.

In [ ]:
from torch.functional import Tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                          shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
# 이미지 몇 개 확인
import matplotlib.pyplot as plt
import numpy as np

# 이미지 보여주기 위한 함수
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))

# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images, labels = dataiter.next()

# 이미지 보여주기
imshow(torchvision.utils.make_grid(images))
# 정답(label) 출력
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## 합성곱 신경망(CNN) 정의하기

In [ ]:
# 이전에 배웠던 신경망 섹션에서 신경망을 복사하고, 
# (기존에 1채널 이미지만 처리하던 것 대신) 3채널 이미지를 처리할 수 있도록 수정합니다.

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

## 손실함수와 Optimizer 정의

이제, 분류에 대한 교차 엔트로피 손실(Cross-Entropy loss)과 momentum을 갖는 SGD를 사용합니다.

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 신경망 학습

데이터를 반복해서 신경망에 입력으로 제공하고, 최적화(Optimize)만 하면 됩니다.

In [ ]:
for epoch in range(2): # 데이터셋을 수차례 반복
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 입력을 받은 후
        inputs, labels = data
        
        # Variable로 감싸고
        inputs, labels = Variable(inputs), Variable(labels)

        # 변화도 매개변수를 0으로 만든 후
        optimizer.zero_grad()

        # 학습 + 역전파 + 최적화
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계 출력 
        running_loss += loss.data
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % 
                  (epoch + 1, i + 1, running_loss/2000))
            running_loss = 0.0

print('Finished Training')

## 시험용 데이터로 신경망 검사하기

신경망이 예측한 정답과 진짜 정답(Ground-truth)을 비교하는 방식으로 확인할텐데요, 예측이 맞다면 샘플을 ‘맞은 예측값(Correct predictions)’에 넣겠습니다.

출력은 10개 분류 각각에 대한 값으로 나타납니다. 

어떤 분류에 대해서 더 높은 값이 나타난다는 것은, 신경망이 그 이미지가 더 해당 분류에 가깝다고 생각한다는 것입니다. 

따라서, 가장 높은 값을 갖는 인덱스(index)를 뽑아보겠습니다:

In [ ]:
outputs = net(Variable(images))

_, predicted = torch.max(outputs.data, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [ ]:
# 전체 데이터셋에 대해서
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%'%(100 * correct / total))

In [ ]:
# 어떤 걸 잘 분류하고, 어떤 걸 못했는지
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

## GPU에서 학습하기

Tensor를 GPU로 옮겼던 것처럼, 신경망을 GPU로 옮길 수 있습니다. 

이렇게 하면 모든 모듈에 대해서 재귀적으로 처리하며 매개변수와 버퍼들을 CUDA tensor로 변환하게 됩니다:


In [ ]:
net.cuda()

모든 단계에서 입력(input)과 정답(target)도 GPU로 보내야 한다는 것도 기억하셔야 합니다:

In [ ]:
inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

CPU와 비교했을 때 어마어마한 속도 차이가 나지 않는 것은 왜 그럴까요? 

그 이유는 바로 신경망이 너무 작기 때문입니다.